In [11]:
import os

In [12]:
%pwd

'C:\\Users\\91996\\Text-Summarizer'

In [13]:
os.chdir(r'C:\Users\91996\Text-Summarizer')

In [14]:
# !pip install --upgrade accelerate
# !pip uninstall -y transformers accelerate
# !pip install transformers accelerate
# !pip install --upgrade datasets
# !pip install evaluate
# !pip install --upgrade transformers accelerate
# !pip install --upgrade transformers


In [15]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir : Path
    data_path : Path
    model_ckpt : Path 
    num_train_epochs : int
    warmup_steps : int
    per_device_train_batch_size : int
    per_device_eval_batch_size : int
    weight_decay : float
    logging_steps : int
    # evaluation_strategy : str
    eval_steps : int
    save_steps : float
    gradient_accumulation_steps : int

In [16]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories
from box import ConfigBox

In [17]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = ConfigBox(read_yaml(config_filepath))
        self.params = ConfigBox(read_yaml(params_filepath))

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            per_device_eval_batch_size = params.per_device_eval_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            # evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps            
        )
        return model_trainer_config
    
        

In [18]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch


In [ ]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model = model_pegasus)

        # loading data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        # Use only first 100 samples from each split
        train_dataset = dataset_samsum_pt["train"].select(range(100))
        eval_dataset = dataset_samsum_pt["test"].select(range(100))

        trainer_args = TrainingArguments(
            output_dir = self.config.root_dir,
            # data_path = self.config.data_path,
            # model_ckpt = self.config.model_ckpt,
            num_train_epochs = self.config.num_train_epochs,
            warmup_steps = self.config.warmup_steps,
            per_device_train_batch_size = self.config.per_device_train_batch_size,
            per_device_eval_batch_size = self.config.per_device_eval_batch_size,
            weight_decay = self.config.weight_decay,
            logging_steps = self.config.logging_steps,
            # evaluation_strategy = self.config.evaluation_strategy,
            eval_steps = self.config.eval_steps,
            save_steps = self.config.save_steps,
            gradient_accumulation_steps = self.config.gradient_accumulation_steps
        )

        trainer = Trainer(
            model=model_pegasus,
            args=trainer_args,
            tokenizer=tokenizer,
            data_collator=seq2seq_data_collator,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset
        )
        
        trainer.train()

        # save  model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum.model"))

        # save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer.model"))


: 

In [ ]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()

except Exception as e:
    raise e

[2025-04-12 15:19:50,656: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-12 15:19:50,677: INFO: common: yaml file: params.yaml loaded successfully]


[2025-04-12 15:19:50,680: INFO: common: created directory at: artifacts]
[2025-04-12 15:19:50,686: INFO: common: created directory at: artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\91996\AppData\Local\Temp\ipykernel_20432\2741452366.py:35: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
